# Halifax Regional Municipality - Terrain Products for Fire Hazard Mapping

Program: TerrainAnalysis.ipynb  
Programmer: Brian Gauthier  
Purpose: This notebook performs terrain analysis on a given dataset to compute slope, aspect, and elevation models.  
Date: April 21, 2025

### Import python modules

In [ ]:
import ee
import sys
import os
import geemap

### Authenticate and Initialize Google Earth Engine and geemap

In [ ]:
#ee.Authenticate()
ee.Initialize(project='bgcloud87')
geemap.ee_initialize()

### Create Map Object

In [ ]:
map = geemap.Map()

### Load administrative boundaries dataset

In [ ]:
dataset = ee.FeatureCollection("FAO/GAUL/2015/level2")

### Investigate the dataset so you know what values to work with

In [ ]:
# Get the first feature in the dataset
first_feature = dataset.first()

# Get the properties (field names)
field_names = first_feature.propertyNames().getInfo()
print(field_names)


In [ ]:
# Get a list of unique Province level values (ADM1 in this case)
adm1_names = dataset.aggregate_array('ADM1_NAME').getInfo()
print(set(adm1_names))  # Convert to set to remove duplicates


In [ ]:
# Get a list of unique County level values (ADM2_NAME in this case)
adm2_names = dataset.aggregate_array('ADM2_NAME').getInfo()
print(set(adm2_names))  # Convert to set to remove duplicates

### Filter for your area of inteterest (HRM)

In [ ]:
# Filter for Nova Scotia
ns = dataset.filter(ee.Filter.eq('ADM1_NAME', 'Nova Scotia / Nouvelle-Écosse'))

print(ns.size().getInfo())

In [ ]:
# Filter for Halifax, Nova Scotia
hali = ns.filter(
    ee.Filter.eq('ADM2_NAME', 'Halifax')
)

# Check the size of the filtered collection
print(hali.size().getInfo())

In [ ]:
# Add to map
map.addLayer(hali.geometry(), {'color': 'red'}, "HRM Boundary")
map.setCenter(-63.106018, 44.871443, 7)

map



### Remove non-contiguous entities (Sable Island)

In [ ]:
# Define Sable Island's approximate boundary as a buffer around its coordinates
sable_island = ee.Geometry.Point([-59.9061, 43.9336]).buffer(100000)  # 100km buffer

# Get the geometry of Halifax
hali_geom = hali.geometry()

# Subtract Sable Island from Halifax's geometry
hali_fixed_geom = hali_geom.difference(sable_island)

# Convert back to a FeatureCollection
hali_fixed = ee.FeatureCollection(ee.Feature(hali_fixed_geom))

# Add to the map
map.addLayer(hali_fixed, {'color': 'blue'}, "HRM (Without Sable Island)")
map.setCenter(-63.106018, 44.871443, 8)
map 


### Split the HRM into 4 equal sections based on longitude

In [ ]:
# Get the bounds of the hali_fixed area
hali_bounds = hali_fixed.bounds()

# Print the bounds (coordinates of the bounding box)
bounds = hali_bounds.getInfo()
print("Bounds of hali_fixed:", bounds)

# Coordinates of the bounding box
coordinates = bounds['coordinates'][0]

# Extract all the longitudes and latitudes
longitudes = [coord[0] for coord in coordinates]
latitudes = [coord[1] for coord in coordinates]

# Calculate the minimum and maximum longitudes and latitudes
min_long = min(longitudes)
max_long = max(longitudes)
min_lat = min(latitudes)
max_lat = max(latitudes)

# Calculate the step size for splitting the longitude range into four equal sections
step = (max_long - min_long) / 4

# Define the three split longitudes (boundaries) to create four equal sections
split_longitude_1 = min_long + step
split_longitude_2 = min_long + 2 * step
split_longitude_3 = min_long + 3 * step

# Print the split longitudes
print("Split Longitudes:", split_longitude_1, split_longitude_2, split_longitude_3)

# Define the four new polygons by cutting the original polygon at the three split longitudes
# West region (1st section)
west_region = ee.Geometry.Polygon([
    [
        [min_long, min_lat],                # Southwest corner
        [split_longitude_1, min_lat],       # Southeast corner
        [split_longitude_1, max_lat],       # Northeast corner
        [min_long, max_lat],                # Northwest corner
        [min_long, min_lat]                 # Closing back to Southwest corner
    ]
])

# Centre-West region (2nd section)
centre_west_region = ee.Geometry.Polygon([
    [
        [split_longitude_1, min_lat],       # Southwest corner
        [split_longitude_2, min_lat],       # Southeast corner
        [split_longitude_2, max_lat],       # Northeast corner
        [split_longitude_1, max_lat],       # Northwest corner
        [split_longitude_1, min_lat]        # Closing back to Southwest corner
    ]
])

# Centre-East region (3rd section)
centre_east_region = ee.Geometry.Polygon([
    [
        [split_longitude_2, min_lat],       # Southwest corner
        [split_longitude_3, min_lat],       # Southeast corner
        [split_longitude_3, max_lat],       # Northeast corner
        [split_longitude_2, max_lat],       # Northwest corner
        [split_longitude_2, min_lat]        # Closing back to Southwest corner
    ]
])

# East region (4th section)
east_region = ee.Geometry.Polygon([
    [
        [split_longitude_3, min_lat],       # Southwest corner
        [max_long, min_lat],                # Southeast corner
        [max_long, max_lat],                # Northeast corner
        [split_longitude_3, max_lat],       # Northwest corner
        [split_longitude_3, min_lat]        # Closing back to Southwest corner
    ]
])

# Add the four regions to the map with new names
map.addLayer(west_region, {'color': 'blue'}, 'West Region')
map.addLayer(centre_west_region, {'color': 'red'}, 'Centre-West Region')
map.addLayer(centre_east_region, {'color': 'green'}, 'Centre-East Region')
map.addLayer(east_region, {'color': 'yellow'}, 'East Region')

# Clip the regions by intersecting them with the HRM boundary
west_clip = west_region.intersection(hali_fixed)
centre_west_clip = centre_west_region.intersection(hali_fixed)
centre_east_clip = centre_east_region.intersection(hali_fixed)
east_clip = east_region.intersection(hali_fixed)

# Add the clipped regions to the map
map.addLayer(west_clip, {'color': 'blue'}, 'West Clipped')
map.addLayer(centre_west_clip, {'color': 'red'}, 'Centre-West Clipped')
map.addLayer(centre_east_clip, {'color': 'green'}, 'Centre-East Clipped')
map.addLayer(east_clip, {'color': 'yellow'}, 'East Clipped')


# Set the map center and zoom level
map.centerObject(hali_fixed, 8)

# Display the map
map

#### Pull Sentinel-2 Satellite imagery - Focus on "Centre West" clipped section

In [ ]:
# load Sentinel-2 image collection
s2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(centre_west_clip) \
    .filterDate('2024-06-01', '2024-10-31')

# Load cloud probability dataset
clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
    .filterBounds(centre_west_clip) \
    .filterDate('2024-06-01', '2024-10-31')

# Function to mask clouds based on cloud probability (< 30%)
def mask_clouds(image):
    cloud_prob = clouds.filter(ee.Filter.equals('system:index', image.get('system:index'))).first().select('probability')
    mask = cloud_prob.lt(30)  # Keep pixels with less than 30% cloud probability
    return image.updateMask(mask)

# Apply the cloud mask to the collection
s2_clean = s2.map(mask_clouds)

# reduce collection to a single image
s2_filtered = s2_clean.median()

# Clip the image to the boundary of hali_fixed
s2_clipped = s2_filtered.clip(centre_west_clip)

# Visualization parameters for RGB (Red, Green, Blue)
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']  # Red, Green, Blue bands
}

# Add the image to the map
map.addLayer(s2_clipped, vis_params, 'CW Sentinel-2 Image (Clipped)')
map

### Terrain Retrieval & Visualization

In [ ]:
# Retrieve and clip the SRTM DEM data
srtm = ee.Image("USGS/SRTMGL1_003")

# Clip the DEM to the AOI
cw_srtm_clip = srtm.clip(centre_west_clip)

# Adjusted visualization parameters for DEM
dem_vis_params = {
    'min': 0,         # Minimum value for elevation (change if needed)
    'max': 200,       # Maximum value for elevation (adjust based on expected range)
    'palette': ['blue', 'green', 'yellow', 'red']  # Color gradient for visualization
}

# Add the DEM layer to the map
map.centerObject(centre_west_clip, 10)
map.addLayer(cw_srtm_clip, dem_vis_params, 'CW SRTM DEM')
map



In [ ]:
# Load NASA DEM and clip to Centre-West
nasa_dem = ee.Image("NASA/NASADEM_HGT/001")
cw_nasa_dem = nasa_dem.select('elevation').clip(centre_west_clip)

# Compute min/max elevation dynamically
stats = cw_nasa_dem.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=centre_west_clip,
    scale=30,
    bestEffort=True
)
min_elev = stats.get('elevation_min').getInfo()
max_elev = stats.get('elevation_max').getInfo()

# Visualization parameters with dynamic scaling
dem_vis_params = {
    'min': min_elev,
    'max': max_elev,
    'palette': ['blue', 'green', 'yellow', 'red']
}


# Center map and add layers
map.centerObject(centre_west_clip, 10)
map.addLayer(cw_nasa_dem, dem_vis_params, 'CW NASA DEM')

map


## Terrain Products & Hazard Layers

### Hillshade

In [ ]:
# Compute and add hillshade for better terrain visualization
hillshade = ee.Terrain.hillshade(cw_nasa_dem)

map.centerObject(centre_west_clip, 9)
map.addLayer(hillshade, {'min': 0, 'max': 255}, 'Hillshade')

map

### Slope

In [ ]:
# Create slope variable from DEM
slope = ee.Terrain.slope(cw_nasa_dem)

# Calculate Min/Max slope values (GEE default unit is degrees)
slope_stats = slope.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=centre_west_clip,
    scale=30,  # Adjust scale based on your DEM resolution
    bestEffort=True
)
print(slope_stats.getInfo())  # Print slope min/max values for debugging

def fire_hazard_slope(slope_image):
    """
    Assigns fire hazard based on slope steepness:
    - 0-5°: Very Low hazard
    - 5-10°: Low hazard
    - 10-25°: Medium hazard
    - 25-31°: High hazard
    - >31°: Very High hazard
    """
    # Define the hazard levels
    very_low_hazard = slope_image.lte(5).multiply(0)  # very low hazard for slopes <= 5 degrees
    low_hazard = slope_image.gt(5).And(slope_image.lte(10)).multiply(1)  # Low hazard
    medium_hazard = slope_image.gt(10).And(slope_image.lte(25)).multiply(2)  # Medium hazard
    high_hazard = slope_image.gt(25).And(slope_image.lte(31)).multiply(3)  # High hazard
    very_high_hazard = slope_image.gt(31).multiply(4)  # Very High hazard

    # Combine all hazard levels into one image
    fire_hazard_slope_map = very_low_hazard.add(low_hazard).add(medium_hazard).add(high_hazard).add(very_high_hazard)
    
    
    return fire_hazard_slope_map

# Apply the fire hazard function to the slope image
fire_hazard_slope_map = fire_hazard_slope(slope)

# Visualization of fire hazard (adjusted visualization based on new hazard values)
fire_hazard_vis = {
    'min': 0,
    'max': 4,
    'palette': ['blue', 'yellow', 'orange', 'red', '#8B0000'],  # Low -> High hazard colors
}

# Add the fire hazard layer to the map
map.addLayer(fire_hazard_slope_map, fire_hazard_vis, "Fire Hazard by Slope")

# Define the legend dictionary with keys (labels) and colors
legend_dict = {
    'Very Low Hazard (0-5°)': '#0000FF',  # Blue
    'Low Hazard (5-10°)': '#FFFF00',  # Yellow
    'Medium Hazard (10-25°)': '#FFA500',  # Orange
    'High Hazard (25-31°)': '#FF0000',  # Red
    'Very High Hazard (>31°)': '#8B0000'  # Dark Red
}


# Add the legend to the map
map.add_legend(title="Fire Hazard by Slope", legend_dict=legend_dict)

# Center the map around the region
map.centerObject(centre_west_clip, 9)

map


### Aspect

In [ ]:
# Compute aspect from the DEM
aspect = ee.Terrain.aspect(cw_nasa_dem)

def fire_hazard_aspect(aspect_image):
    """
    Assigns fire hazard based on aspect and degree range:
    - South (135° - 225°): Extremely High Hazard
    - North (315° - 45°): Low Hazard
    - East (45° - 135°): Medium Hazard
    - West (225° - 315°): High Hazard
    """
    # Define aspect ranges for each cardinal direction
    extremely_high_hazard = aspect_image.gt(135).And(aspect_image.lte(225)).multiply(4)  # Very High hazard (Dark Red)
    low_hazard = (aspect_image.lt(45).Or(aspect_image.gt(315))).multiply(1)  # Low hazard (Yellow)
    medium_hazard = aspect_image.gt(45).And(aspect_image.lte(135)).multiply(2)  # Medium hazard (Orange)
    high_hazard = aspect_image.gt(225).And(aspect_image.lte(315)).multiply(3)  # High hazard (Red)


    fire_hazard_map = low_hazard.add(medium_hazard).add(high_hazard).add(extremely_high_hazard)
    
    return fire_hazard_map

# Apply the updated aspect fire hazard function
fire_hazard_aspect_map = fire_hazard_aspect(aspect)

# Visualization of fire hazard (from low to high hazard)
fire_hazard_aspect_vis = {
    'min': 1,
    'max': 4,
    'palette': ['#FFFF00', '#FFA500', '#FF0000', '#8B0000'],  # Low -> High hazard colors (Yellow, Orange, Red, Dark Red)
}

# Center the map and add the aspect layer
map.centerObject(centre_west_clip, 9)
map.addLayer(fire_hazard_aspect_map, fire_hazard_aspect_vis, 'Fire Hazard by Aspect')

# Define the legend dictionary with keys (labels) and colors
legend_dict = {
    'Low Hazard (N, 315° - 45°)': '#FFFF00',  # Yellow
    'Medium Hazard (E, 45° - 135°)': '#FFA500',  # Orange
    'High Hazard (W, 225° - 315°)': '#FF0000',  # Red
    'Extremely High Hazard (S, 135° - 225°)': '#8B0000',  # Dark Red
}

# Add the legend to the map
map.add_legend(title="Fire Hazard by Aspect", legend_dict=legend_dict)

# Display the map
map



### Elevation

In [ ]:
# Load the DEM (cw_nasa_dem) for Nova Scotia
elevation = cw_nasa_dem

# Reclassify the elevation into fire hazard classes based on the ranges
def elevation_fire_hazard(elevation_image):
    """
    Classify the fire hazard based on elevation:
    - 10-100 meters (Extremely High hazard - 5)
    - 100-200 meters (High hazard - 4)
    - 200-300 meters (Medium hazard - 3)
    - 300-400 meters (Low hazard - 2)
    - >400 meters (Extremely Low hazard - 1)
    - 0 meters (No Data hazard - 0)
    """
    # Define the classification for each range
    extremely_high = elevation_image.gte(10).And(elevation_image.lt(100)).multiply(5)
    high = elevation_image.gte(100).And(elevation_image.lt(200)).multiply(4)
    medium = elevation_image.gte(200).And(elevation_image.lt(300)).multiply(3)
    low = elevation_image.gte(300).And(elevation_image.lt(400)).multiply(2)
    extremely_low = elevation_image.gte(400).multiply(1)

    # Handle No Data areas or 0 elevation
    no_data = elevation_image.eq(0).multiply(0)  # Assign 0 for No Data (0m elevation)

    # Combine all the classes, including No Data
    fire_hazard_map = extremely_high.add(high).add(medium).add(low).add(extremely_low).add(no_data)
    
    return fire_hazard_map

# Apply the classification function
elevation_fire_hazard_map = elevation_fire_hazard(elevation)

# Visualization parameters for fire hazard (low to high hazard)
elevation_fire_hazard_vis = {
    'min': 0,  # Now including the No Data category
    'max': 5,
    'palette': ['#808080', '#0000FF', '#FFFF00', '#FFA500', '#FF4500', '#8B0000'],  # Added gray for No Data
}

# Center the map around Nova Scotia
map.centerObject(cw_nasa_dem, 9)

# Add the fire hazard layer to the map
map.addLayer(elevation_fire_hazard_map, elevation_fire_hazard_vis, 'Fire Hazard based on Elevation')

# Define the legend dictionary with keys (labels) and colors
legend_dict = {
    'No Data (0 meters)': '#808080',  # Gray for No Data areas (0 meters)
    'Extremely Low Hazard (>400 meters)': '#0000FF',  # Blue
    'Low Hazard (300-400 meters)': '#FFFF00',  # Yellow
    'Medium Hazard (200-300 meters)': '#FFA500',  # Orange
    'High Hazard (100-200 meters)': '#FF4500',  # Red-Orange
    'Extremely High Hazard (10-100 meters)': '#8B0000',  # Dark Red
}

# Add the legend to the map
map.add_legend(title="Fire Hazard by Elevation", legend_dict=legend_dict)

# Display the map
map
